In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import re

In [ ]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [ ]:
sys_msg = "You are an articulate, curteous, lovable clerk in a store that sells clothing."
products = {
    r"\b[Bb]elts?\b": ["belts", 25.99],
    r"\b[Ss]hoes?|[Ff]ootwear|[Kk]icks\b": ["footwear", 49.99],
    r"\b[Tt]ies?\b": ["ties", 23.00]
}

In [ ]:
def chat(message, history):

    relevant_system_message = sys_msg
    for product in products:
        if re.search(product, message):
            relevant_system_message += f"\nThe store does have {products[product][0]}, they only cost ${products[product][1]} today!"
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
sys_msg += "\nIf the customer asks for hats, you should respond that hats are currently sold out, \
because they were so popular. We'll restock within a couple weeks."

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()